In [1]:
# the cards that are highlighted have the following class (in hand)
# class = card zoomable highlight handy
# the cards in hand that are not highlighted are as follows:
# class = card zoomable handy
# cards on board that are not highlighted
# card zoomable mappy
# cards on board controlled by opponent
# card zoomable mappy opponent-controlled
# cards on board you control that are highlighted
# card zoomable highlight mappy
# choice of taking an action (e.g. attacking, discarding a card, summoning a card, etc.)
# action-choice
# this shows up when attacking an enemy, 
# class= "chooseable-space-overlay" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;"
# the background color when an empty tile is highlighted 
# class="space-placeholder" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;
# the background color when a tile is not highlighted
# style="background-color: rgba(0, 0, 0, 0); cursor: default;
# the buttons on the left side, go to next phase, etc.
# button class="control-panel-content-actions-action"
# empty spaces 
# class="space-placeholder-container"
# cards that opponent controls when you can force them
# card zoomable highlight mappy opponent-controlled

In [2]:
# !pip install webdriver_manager

In [3]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
np.__version__

'1.19.5'

In [4]:

df = pd.read_excel('../sw_alt_password.xlsx')

In [5]:
password = df['password'][1]
email = df['Email'][1]

In [6]:
factions = ['Breakers', 'Cave Goblins', 'Cloaks', 'Crimson Order', 'Eternal Council', 'Fallen Kingdom'
           , 'Fungal Dwarves', 'High Elves', 'Mountain Vargath', 'Obsidian Dwarves',
           'Phoenix Elves', 'Polar Dwarves', 'Sand Goblins', 'Savanna Elves',
           'Shadow Elves', 'Skyspear Avians', 'Swamp Orcs', 'Tundra Orcs',
           'Vanguards', 'Wayfarers']

In [7]:



def start_game(singleplayer, faction_index, opposing_index, email, password, driver):
    url = 'https://www.plaidhatgames.com/accounts/login/?next=/redirect-swo'
    driver.get(url)



    time.sleep(5)
    login = driver.find_element_by_id('id_login')
    login.click()
    login.send_keys(email)
    time.sleep(.5)
    pw = driver.find_element_by_id('id_password')
    pw.click()
    pw.send_keys(password)
    time.sleep(.05)
    button = driver.find_element_by_class_name('primaryAction')
    button.click()
    time.sleep(15)
    buttons = driver.find_elements_by_class_name('menu-button-text-container')
    if singleplayer==True:
        buttons[1].click()
        time.sleep(3)
        # change some settings
        settings = driver.find_element_by_class_name('settings-button-left')
        settings.click()
        time.sleep(1)
        # auto dismiss events (can cause issues otherwise)
        autodismiss_events = driver.find_element_by_id('autodismiss_events')
        autodismiss_events.click()
        time.sleep(.05)
        # turn on attack friendly
        attack_friendly = driver.find_element_by_id('attack_friendly')
        attack_friendly.click()
        time.sleep(.05)
        # turn off the sound
        sound = driver.find_elements_by_id('text-size-input')
        move = ActionChains(driver)
        move.click_and_hold(sound[0]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)
        move.click_and_hold(sound[1]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)

        back_button = driver.find_element_by_class_name('full-overlay-back-button-img')
        back_button.click()
        time.sleep(.5)
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[3].click()
        time.sleep(3)
        buttons = driver.find_elements_by_class_name('use-deck-button')
        buttons[faction_index].click()
        time.sleep(1)
        buttons = driver.find_elements_by_class_name('use-deck-button')
        buttons[opposing_index].click()
    else:
        buttons[0].click()



    

In [8]:
# function that determines whether it's your turn
def is_turn(driver):
    time.sleep(.5)
    try:
        driver.find_element_by_class_name('control-panel-content-actions-action')
        return True
    except:
        return False

In [9]:
# driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(executable_path=r"C:\\Users\\steve\\.wdm\\drivers\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe")
# use this when you need a new version of chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
start_game(singleplayer=True, faction_index=factions.index('Tundra Orcs'), opposing_index=factions.index('Polar Dwarves'),
          email=email, password=password, driver=driver)
print(is_turn(driver))

False


In [ ]:
# driver.page_source

In [124]:
# returns outer html for each space on the map
def get_board_spaces(driver):
    all_spaces = []
    for col in range(6):
        single_col_spaces = []
        for row in range(8):
            space_html = driver.find_element_by_id(f'map-space-{col}-{row}').get_attribute('outerHTML')
            if 'space-placeholder' in space_html:
                friendly = 0
                occupied = False
                card_abilities = ['']
                card_classes = ['']
            else:
                occupied = True
                
                if 'opponent' in space_html:
                    friendly = -1
            
                else:
                    friendly = 1
                    
                card = driver.find_element_by_id(f'map-space-{col}-{row}')
                card.click()
                time.sleep(.05)
                               
    
                zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
        
                
                
                
                if 'zoom-card-ability' in zoom_html:         
                    html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
                    regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
                    card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
                    
                    html_card_classes = re.findall('<div class="zoom-card-extras-unit">[^<]+</div>',zoom_html)
                    regex_card_classes = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-unit">'), re.escape('</div>')))
                    card_classes = regex_card_classes.findall(html_card_classes[0])
#                     card_classes = card_classes[0]
                    
                    
                    if 'Structure' in zoom_html:
                        card_classes.append('Structure')
                    print(card_classes)
                    
                    
                else:
                    card_abilities = ['']
                    card_classes = ['Structure']
                    print(card_classes)

                elem = driver.find_element_by_id('zoom-card')
                ac = ActionChains(driver)
                ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
                time.sleep(.05)
                               
            
            
            # card-name
            if 'card-name' in space_html:
                html_card_name = re.findall('alt=[^<]+><div class="card-name">',space_html)
                regex_card_name = re.compile('{}(.*){}'.format(re.escape('alt='), re.escape('><div class="card-name">')))
                card_name = regex_card_name.findall(html_card_name[0])
                card_name = card_name[0]
            else:
                card_name = ''
            
            # card-strength
            if 'card-strength' in space_html:
                html_card_strength = re.findall('<div class="card-strength">[^<]+</div>',space_html)
                regex_card_strength = re.compile('{}(.*){}'.format(re.escape('<div class="card-strength">'), re.escape('</div>')))
                card_strength = regex_card_strength.findall(html_card_strength[0])
                card_strength = card_strength[0]
            else:
                card_strength=0
            
            # card-life
            if 'card-life' in space_html:
                html_card_life = re.findall('<div class="card-life[^<]+</div>',space_html)
                regex_card_life = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
                card_life = regex_card_life.findall(html_card_life[0])
                card_life = card_life[0]
            else:
                card_life=0
            # boost
            if 'boost' in space_html:
                html_card_boost = re.findall('<div class="boost">[^<]+</div>',space_html)
                regex_card_boost = re.compile('{}(.*){}'.format(re.escape('<div class="boost">'), re.escape('</div>')))
                boost = regex_card_boost.findall(html_card_boost[0])
                boost = boost[0]
            else:
                boost=0
            # card-abilities-ability
#             if 'card-abilities-ability' in space_html:
#                 html_card_abilities = re.findall('<div class="card-abilities-ability">[^<]+</div>',space_html)
#                 regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="card-abilities-ability">'), re.escape('</div>')))
#                 card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
#             else:
#                 card_abilities = ['']
            # append to the column list
            single_col_spaces.append([occupied, friendly, card_name, card_strength, card_life, boost, card_abilities,
                                      card_classes, row, col])
        
        # append to columns to all_spaces so it is a 2 dimensional list (can maybe perform convolutions?)
        all_spaces.append(single_col_spaces)
            
    return all_spaces        
            

In [125]:
spaces = get_board_spaces(driver)
spaces

['Summoner']
['Common', 'Structure']
['Structure']
['Common', 'Structure']
['Common']
['Summoner']
['Common']
['Common']
['Structure']


[[[False, 0, '', 0, 0, 0, [''], [''], 0, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 0]],
 [[False, 0, '', 0, 0, 0, [''], [''], 0, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 1]],
 [[True,
   -1,
   '"Svara"',
   '3',
   '12',
   0,
   ['Structural Shift'],
   ['Summoner'],
   0,
   2],
  [True,
   -1,
   '"Ice Golem"',
   '2',
   '5',
   0,
   ['Living Gate', 'Mobile Structure', 'Slow'],
   ['Common', 'Structure'],
   1,
   2],
  [True, -1, '"gate"', 0,

In [86]:
def get_active_events(driver):
    try:
        active_html = driver.find_element_by_class_name('active-display-opponent').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        opponent_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        opponent_active_events = ['']
    try:
        active_html = driver.find_element_by_class_name('active-display-mine').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        friendly_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        friendly_active_events = ['']
    return opponent_active_events, friendly_active_events

In [88]:
opponent_active_events, friendly_active_events = get_active_events(driver)
opponent_active_events, friendly_active_events

([], ['Insight', 'Learn'])

In [122]:
# function to obtain amount of magic, cards in hand and deck
def get_resources(driver):
    user_frames = driver.find_elements_by_class_name('user-framed-stuff')
    enemy_frames = user_frames[0].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',enemy_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    enemy_magic = regex_magic.findall(html_magic[0])
    enemy_magic = enemy_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',enemy_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    enemy_hand = regex_hand.findall(html_hand[0])
    enemy_hand = enemy_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',enemy_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    enemy_draw = regex_draw.findall(html_draw[0])
    enemy_draw = enemy_draw[0]
    
    friendly_frames = user_frames[1].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',friendly_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    friendly_magic = regex_magic.findall(html_magic[0])
    friendly_magic = friendly_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',friendly_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    friendly_hand = regex_hand.findall(html_hand[0])
    friendly_hand = friendly_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',friendly_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    friendly_draw = regex_draw.findall(html_draw[0])
    friendly_draw = friendly_draw[0]
    return [int(enemy_magic), int(enemy_hand), int(enemy_draw), int(friendly_magic), int(friendly_hand), int(friendly_draw)]


In [123]:
resources = get_resources(driver)
resources

[3, 5, 21, 3, 5, 25]

In [170]:
def get_hand(driver):  
    hand_buttons = driver.find_elements_by_class_name('framed-hand')
    # click on the second hand button (your own hand)
    hand_buttons[1].click()
    time.sleep(.05)
    hand_html = driver.find_element_by_class_name('summoner-wars-table-hand').get_attribute('outerHTML')
    html_card_names = re.findall('class="card-name">[^<]+</div>',hand_html)
    regex_card_names = re.compile('{}(.*){}'.format(re.escape('class="card-name">'), re.escape('</div>')))
    card_names = [regex_card_names.findall(html_card_names[i])[0] for i in range(len(html_card_names))]
    html_card_costs = re.findall('class="card-cost[^"]*"><span>[^<]+</span>',hand_html)
    regex_card_costs = re.compile('{}(.*){}'.format(re.escape('<span>'), re.escape('</span>')))
    card_costs = [int(regex_card_costs.findall(html_card_costs[i])[0]) for i in range(len(html_card_costs))]
    return card_names, card_costs

In [171]:
card_names, card_costs = get_hand(driver)
card_names, card_costs

(['Witch', 'Witch', 'gate', 'Splack', 'Vinemancer'], [1, 1, 0, 4, 1])

In [ ]:
def get_discard(driver):

In [ ]:
def get_deck(driver):

In [ ]:
def get_enemy_discards_remaining_cards(driver)

In [59]:
# returns card name, class, faction, event phase, ability names, ability text, strength, life, magic, card count
def get_card_attributes(driver, factions):
    card_data = []
    card_name_list = []
    for faction in factions:
        # select faction
        faction_button = driver.find_element_by_css_selector(f'[alt="{faction}"]')
        faction_button.click()
        # click continue
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[1].click()
        time.sleep(1)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[0].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(1)
        card_data.append(card_info)
        
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[1].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(2)
        card_data.append(card_info)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[2].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(5)
        card_data.append(card_info)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[3].click()
        card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
        card_info.append(faction)
        card_info.append(5)
        card_data.append(card_info)
        time.sleep(.05)
        
        
        
        card_counts_in_faction = []
        card_counts = driver.find_elements_by_class_name('card-quantity')
        for count in card_counts:
            if 'x0' not in count.get_attribute('outerHTML'):
                card_counts_in_faction.append(count)
        for card_count in card_counts_in_faction:
            ac = ActionChains(driver)
            ac.move_to_element(card_count).move_by_offset(-20, 0).click().perform()
            card_info, card_name_list = get_card_attributes_zoomed_in_card(driver, card_name_list)
            # as long as card has not already been added
            if card_info:
                # obtain the card count
                html_card_count = re.findall('x[^<]+</div>',card_count.get_attribute('outerHTML'))
                regex_card_count = re.compile('{}(.*){}'.format(re.escape('x'), re.escape('</div>')))
                count = regex_card_count.findall(html_card_count[0])
                count = count[0]
                # append faction and card count
                card_info.append(faction)
                card_info.append(count)
                # append the card info as long as it's a new card
                card_data.append(card_info)
            time.sleep(.05)
            
            
        # go back
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[2].click()
        time.sleep(1)
        
    return card_data
        

In [63]:
def get_card_attributes_zoomed_in_card(driver, card_name_list):
    zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
    
    html_card_name = re.findall('<div class="zoom-card-name-full">[^<]+</div>',zoom_html)
    regex_card_name = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-name-full">'), re.escape('</div>')))
    card_name = regex_card_name.findall(html_card_name[0])
    card_name = card_name[0]
    print(card_name)
    if card_name not in card_name_list:
        card_name_list.append(card_name)
        
        if 'zoom-card-ability-name' in zoom_html:
            html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
            regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
            card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
            html_card_abilities_text = re.findall('<div class="zoom-card-ability-text"><span>[^<]+</span>',zoom_html)
            regex_card_abilities_text = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-text"><span>'), re.escape('</span>')))
            card_abilities_text = [regex_card_abilities_text.findall(html_card_abilities_text[i])[0] for i in range(len(html_card_abilities_text))]

        else:
            card_abilities = [None]
            card_abilities_text = [None]
            
        card_classes = []
        
        if 'zoom-card-extras-unit' in zoom_html:   
            html_card_classes = re.findall('<div class="zoom-card-extras-unit">[^<]+</div>',zoom_html)
            regex_card_classes = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-unit">'), re.escape('</div>')))
            card_classes = regex_card_classes.findall(html_card_classes[0])
        if 'Structure' in zoom_html:
            card_classes.append('Structure')
            


        if ('place ' in zoom_html or 'Place ' in zoom_html) and ' under ' in zoom_html:
            go_under = True
        else:
            go_under = False
            
        if 'event-text-active' in zoom_html:
            active_event = True
        else:
            active_event = False
            
        if 'zoom-card-extras-event-phase' in zoom_html and 'Structure' not in zoom_html:
            html_event_phase = re.findall('<div class="zoom-card-extras-event-phase">[^<]+</div>',zoom_html)
            regex_event_phase = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-event-phase">'), re.escape('</div>')))
            event_phase = regex_event_phase.findall(html_event_phase[0])
            event_phase = event_phase[0]
        else:
            event_phase = None
            
            
        

        
            
        if 'zoom-card-cost' in zoom_html:
            html_card_cost = re.findall('<div class="zoom-card-cost[^"]*">[^<]+</div>',zoom_html)
            regex_card_cost = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
            card_cost = regex_card_cost.findall(html_card_cost[0])
            card_cost = card_cost[0]
        else:
            card_cost = None
            
        
            
        
        # so only units are selected (for whatever reason events have an empty strength div)
        if 'zoom-card-extras-unit' in zoom_html:
            html_card_strength = re.findall('<div class="zoom-card-strength">[^<]+</div>',zoom_html)
            regex_card_strength = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-strength">'), re.escape('</div>')))
            card_strength = regex_card_strength.findall(html_card_strength[0])
            card_strength = card_strength[0]
        else:
            card_strength = None
        
        # only units and structures should have a life value
        if 'zoom-card-extras-unit' in zoom_html or 'Structure' in zoom_html:
            html_card_life = re.findall('<div class="zoom-card-life[^"]*">[^<]+</div>',zoom_html)
            regex_card_life = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
            card_life = regex_card_life.findall(html_card_life[0])
            card_life = card_life[0]
        else:
            card_life = None



        elem = driver.find_element_by_id('zoom-card')
        ac = ActionChains(driver)
        ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
        time.sleep(.05)

        return [card_name, card_classes, event_phase, active_event, card_abilities, card_abilities_text, 
               go_under, card_cost, card_strength, card_life], card_name_list
    
    else:
        elem = driver.find_element_by_id('zoom-card')
        ac = ActionChains(driver)
        ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
        time.sleep(.05)

        return False, card_name_list

In [64]:
# card_data = get_card_attributes(driver, factions)
# card_data

Tacullu
Mind Control
Deceiver
Mind Witch
Gulldune
Gwalark
Kalal
Deceiver
Mind Witch
Wind Archer
Wind Mage
Blast
Hypnotic Call
Perplexing Tempest
Sneeks
Enrage the Horde
Horde Slinger
Beast Rider
Blarf
Smeg
The Eater
Beast Rider
Clinger
Horde Climber
Horde Slinger
Pile On
Sneak
Unrelenting
Vlox
Out of Shadows
Guild Machinist
Guild Sniper
Admiral
Dagger
Sin-Sin
Guild Gunner
Guild Machinist
Guild Sniper
Rustside Bandit
Hand Cannon
Moto Boots
Smoke Bomb
Baron Daemar
Convert
Royal Warrior
Devout Cultist
Casimira
Sekhmet
Vesna Brana
Devout Cultist
Doom Cultist
Royal Warrior
Sacral Priest
Call Forth
Death Rite
Transfuse
Praefectus Edia
Learn
Citadel Advisor
Mind Knight
Dominus Katu
Dominus Malunar
Dominus Ovi
Ancient Scholar
Citadel Advisor
Enigma Sage
Mind Knight
Insight
Psychic Assault
Seek
Ret-Talus
Sacrificial Pyre
Undead Carrier
Undead Archer
Dragos
Elut-Bal
Gul-Dass
Hellfire Cultist
Undead Archer
Undead Carrier
Undead Warrior
Blood Summon
Hellforged Weapon
Purge
Kuldak
Release Spores
Fu

[['Tacullu',
  ['Summoner'],
  None,
  False,
  ['Mind Capture'],
  ['When this unit attacks an enemy unit, if the damage added would be enough to destroy the target, you may instead ignore that damage and take control of the target.'],
  False,
  None,
  '3',
  '13',
  'Breakers',
  1],
 ['Mind Control',
  [],
  'Summon Phase',
  False,
  [None],
  [None],
  False,
  '0',
  None,
  None,
  'Breakers',
  2],
 ['Deceiver',
  ['Common'],
  None,
  False,
  ['Stupify', 'Engage'],
  ['When an adjacent enemy attacks any card, if 1 or more ',
   'Each time an adjacent enemy unit moves or is forced away from this unit, add 1 damage to that enemy.'],
  False,
  '1',
  '1',
  '4',
  'Breakers',
  5],
 ['Mind Witch',
  ['Common'],
  None,
  False,
  ['Mimic'],
  ['At the start of your Move Phase, this unit may target a common within 3 spaces. This unit has the target’s abilities until the end of your turn.'],
  False,
  '1',
  '3',
  '2',
  'Breakers',
  5],
 ['Gulldune',
  ['Champion'],
  None,

In [67]:
# card_df = pd.DataFrame(card_data,columns=['Card Name', 'Card Classes', 'Event Phase', 'Active Event', 'Abilities', 'Ability Text', 
#                'Under', 'Cost', 'Strength', 'Life', 'Faction', 'Copies'])

In [69]:
# card_df.to_csv('Summoner_Wars_Cards.csv')

In [81]:
# returns list of card names, abilities, active events,
# cards that go under other cards (will be used to create a list of these for one hot encoding)
def list_card_attributes(driver, factions):
    card_name_list = []
    card_ability_list = []
    battlefield_list = []
    under_card_list = []
    active_event_list = []
    summoner_list = []
    common_setup_list = []
    for faction in factions:
        # select faction
        faction_button = driver.find_element_by_css_selector(f'[alt="{faction}"]')
        faction_button.click()
        # click continue
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[1].click()
        time.sleep(1)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[0].click()
        card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list = get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list)
        summoner_list.append(card_name_list[-1])
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[1].click()
        card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list = get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[2].click()
        name, _, _, _, _ = get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list)
        common_setup_list.append(name)
        time.sleep(.05)
        
        cards = driver.find_elements_by_class_name('card-image')
        cards[3].click()
        name, _, _, _, _ = get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list)
        common_setup_list.append(name)
        time.sleep(.05)
        
        
        
        card_counts_in_faction = []
        card_counts = driver.find_elements_by_class_name('card-quantity')
        for count in card_counts:
            if 'x0' not in count.get_attribute('outerHTML'):
                card_counts_in_faction.append(count)
        for card_count in card_counts_in_faction:
            ac = ActionChains(driver)
            ac.move_to_element(card_count).move_by_offset(-20, 0).click().perform()
            card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list = get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list)
            time.sleep(.05)
            
            
        # go back
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[2].click()
        time.sleep(1)
        
    return card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list, summoner_list, common_setup_list
        
        
        
        

In [96]:
def get_card_categories_zoomed_in_card(driver, card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list):
    zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
    
    html_card_name = re.findall('<div class="zoom-card-name-full">[^<]+</div>',zoom_html)
    regex_card_name = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-name-full">'), re.escape('</div>')))
    card_name = regex_card_name.findall(html_card_name[0])
    card_name = card_name[0]
    print(card_name)
    if card_name not in card_name_list:
        card_name_list.append(card_name)
        
        if 'zoom-card-ability-name' in zoom_html:
            html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
            regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
            card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
            for i in card_abilities:
                card_ability_list.append(i)

    #     else:
    #         card_abilities = [None]

    #     html_card_abilities_text = re.findall('<div class="zoom-card-ability-text"><span>[^<]+</span>',zoom_html)
    #     regex_card_abilities_text = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-text"><span>'), re.escape('</span>')))
    #     card_abilities_text = [regex_card_abilities_text.findall(html_card_abilities_text[i])[0] for i in range(len(html_card_abilities_text))]

        if 'zoom-card-extras-unit' in zoom_html:
            battlefield_list.append(card_name)
        elif 'zoom-card-extras-structure' in zoom_html:
            battlefield_list.append(card_name)

        if ('place ' in zoom_html or 'Place ' in zoom_html) and ' under ' in zoom_html:
            under_card_list.append(card_name)

        if 'event-text-active' in zoom_html:
            active_event_list.append(card_name)



    elem = driver.find_element_by_id('zoom-card')
    ac = ActionChains(driver)
    ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
    time.sleep(.05)
    
    return card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list
    
    
    
    

In [97]:
# returns several lists
# card_name_list, card_ability_list, battlefield_list, under_card_list, active_event_list, summoner_list, common_setup_list = list_card_attributes(driver, factions)
# card_name_list

Tacullu
Mind Control
Deceiver
Mind Witch
Gulldune
Gwalark
Kalal
Deceiver
Mind Witch
Wind Archer
Wind Mage
Blast
Hypnotic Call
Perplexing Tempest
Sneeks
Enrage the Horde
Horde Slinger
Beast Rider
Blarf
Smeg
The Eater
Beast Rider
Clinger
Horde Climber
Horde Slinger
Pile On
Sneak
Unrelenting
Vlox
Out of Shadows
Guild Machinist
Guild Sniper
Admiral
Dagger
Sin-Sin
Guild Gunner
Guild Machinist
Guild Sniper
Rustside Bandit
Hand Cannon
Moto Boots
Smoke Bomb
Baron Daemar
Convert
Royal Warrior
Devout Cultist
Casimira
Sekhmet
Vesna Brana
Devout Cultist
Doom Cultist
Royal Warrior
Sacral Priest
Call Forth
Death Rite
Transfuse
Praefectus Edia
Learn
Citadel Advisor
Mind Knight
Dominus Katu
Dominus Malunar
Dominus Ovi
Ancient Scholar
Citadel Advisor
Enigma Sage
Mind Knight
Insight
Psychic Assault
Seek
Ret-Talus
Sacrificial Pyre
Undead Carrier
Undead Archer
Dragos
Elut-Bal
Gul-Dass
Hellfire Cultist
Undead Archer
Undead Carrier
Undead Warrior
Blood Summon
Hellforged Weapon
Purge
Kuldak
Release Spores
Fu

['Tacullu',
 'Mind Control',
 'Deceiver',
 'Mind Witch',
 'Gulldune',
 'Gwalark',
 'Kalal',
 'Wind Archer',
 'Wind Mage',
 'Blast',
 'Hypnotic Call',
 'Perplexing Tempest',
 'Sneeks',
 'Enrage the Horde',
 'Horde Slinger',
 'Beast Rider',
 'Blarf',
 'Smeg',
 'The Eater',
 'Clinger',
 'Horde Climber',
 'Pile On',
 'Sneak',
 'Unrelenting',
 'Vlox',
 'Out of Shadows',
 'Guild Machinist',
 'Guild Sniper',
 'Admiral',
 'Dagger',
 'Sin-Sin',
 'Guild Gunner',
 'Rustside Bandit',
 'Hand Cannon',
 'Moto Boots',
 'Smoke Bomb',
 'Baron Daemar',
 'Convert',
 'Royal Warrior',
 'Devout Cultist',
 'Casimira',
 'Sekhmet',
 'Vesna Brana',
 'Doom Cultist',
 'Sacral Priest',
 'Call Forth',
 'Death Rite',
 'Transfuse',
 'Praefectus Edia',
 'Learn',
 'Citadel Advisor',
 'Mind Knight',
 'Dominus Katu',
 'Dominus Malunar',
 'Dominus Ovi',
 'Ancient Scholar',
 'Enigma Sage',
 'Insight',
 'Psychic Assault',
 'Seek',
 'Ret-Talus',
 'Sacrificial Pyre',
 'Undead Carrier',
 'Undead Archer',
 'Dragos',
 'Elut-Bal',

In [114]:
summoner_list

['Tacullu',
 'Sneeks',
 'Vlox',
 'Baron Daemar',
 'Praefectus Edia',
 'Ret-Talus',
 'Kuldak',
 'Valeria the Just',
 'Sunderved',
 'Balzar',
 'Queen Maldaria',
 'Svara',
 'Krusk',
 'Abua Shi',
 'Selundar',
 'Ava',
 'Mugglug',
 'Grognack',
 'Sera Eldwyn',
 'Celeste']

In [115]:
common_setup_list

[['Tacullu',
  'Mind Control',
  'Deceiver',
  'Mind Witch',
  'Gulldune',
  'Gwalark',
  'Kalal',
  'Wind Archer',
  'Wind Mage',
  'Blast',
  'Hypnotic Call',
  'Perplexing Tempest',
  'Sneeks',
  'Enrage the Horde',
  'Horde Slinger',
  'Beast Rider',
  'Blarf',
  'Smeg',
  'The Eater',
  'Clinger',
  'Horde Climber',
  'Pile On',
  'Sneak',
  'Unrelenting',
  'Vlox',
  'Out of Shadows',
  'Guild Machinist',
  'Guild Sniper',
  'Admiral',
  'Dagger',
  'Sin-Sin',
  'Guild Gunner',
  'Rustside Bandit',
  'Hand Cannon',
  'Moto Boots',
  'Smoke Bomb',
  'Baron Daemar',
  'Convert',
  'Royal Warrior',
  'Devout Cultist',
  'Casimira',
  'Sekhmet',
  'Vesna Brana',
  'Doom Cultist',
  'Sacral Priest',
  'Call Forth',
  'Death Rite',
  'Transfuse',
  'Praefectus Edia',
  'Learn',
  'Citadel Advisor',
  'Mind Knight',
  'Dominus Katu',
  'Dominus Malunar',
  'Dominus Ovi',
  'Ancient Scholar',
  'Enigma Sage',
  'Insight',
  'Psychic Assault',
  'Seek',
  'Ret-Talus',
  'Sacrificial Pyre'

In [ ]:
# tbh I think I preferred when I tried to create a list of each unit along with attributes... that way I can convert
# to a dataframe

In [113]:
active_event_list = ['Hypnotic Call', 'Perplexing Tempest', 'Pile On', 'Unrelenting', 'Out of Shadows',
 'Smoke Bomb', 'Convert', 'Death Rite', 'Learn', 'Insight', 'Psychic Assault', 'Seek', 'Sacrificial Pyre',
 'Stimshrooms', 'Law of Liberty', 'Law of Loyalty', 'Law of Sanctity', 'Battle Leader', 'Ramming Advance',
 'Willpower', 'Spirit of the Phoenix', 'Blinding Flare', 'Divine Retribution', 'Ice Ram', 'Chant of Entangling',
 'Chant of Weaving', 'Glinting Speed', 'Gathering Song', 'Battle Song', 'Sky Assault', 'Emerge', 'For Glory',
 'Brutal Force', 'Freeze', 'Primal Fury', 'Holy Judgment', 'Guarding Spirits', 'Renewed Hope', 'Coordinate',
 'Seek the Horizon', 'Slip By']

['Hypnotic Call',
 'Perplexing Tempest',
 'Pile On',
 'Unrelenting',
 'Out of Shadows',
 'Smoke Bomb',
 'Convert',
 'Death Rite',
 'Learn',
 'Insight',
 'Psychic Assault',
 'Seek',
 'Sacrificial Pyre',
 'Stimshrooms',
 'Law of Liberty',
 'Law of Loyalty',
 'Law of Sanctity',
 'Battle Leader',
 'Ramming Advance',
 'Willpower',
 'Spirit of the Phoenix',
 'Blinding Flare',
 'Divine Retribution',
 'Ice Ram',
 'Chant of Entangling',
 'Chant of Weaving',
 'Glinting Speed',
 'Gathering Song',
 'Battle Song',
 'Sky Assault',
 'Emerge',
 'For Glory',
 'Brutal Force',
 'Freeze',
 'Primal Fury',
 'Holy Judgment',
 'Guarding Spirits',
 'Renewed Hope',
 'Coordinate',
 'Seek the Horizon',
 'Slip By']

In [112]:
battlefield_list = ['Tacullu', 'Deceiver', 'Mind Witch', 'Gulldune', 'Gwalark', 'Kalal', 'Wind Archer',
 'Wind Mage', 'Sneeks', 'Horde Slinger', 'Beast Rider', 'Blarf', 'Smeg', 'The Eater', 'Clinger',
 'Horde Climber', 'Vlox', 'Guild Machinist', 'Guild Sniper', 'Admiral', 'Dagger', 'Sin-Sin', 'Guild Gunner',
 'Rustside Bandit', 'Baron Daemar', 'Royal Warrior', 'Devout Cultist',
 'Casimira',
 'Sekhmet',
 'Vesna Brana',
 'Doom Cultist',
 'Sacral Priest',
 'Praefectus Edia',
 'Citadel Advisor',
 'Mind Knight',
 'Dominus Katu',
 'Dominus Malunar',
 'Dominus Ovi',
 'Ancient Scholar',
 'Enigma Sage',
 'Ret-Talus',
 'Undead Carrier',
 'Undead Archer',
 'Dragos',
 'Elut-Bal',
 'Gul-Dass',
 'Hellfire Cultist',
 'Undead Warrior',
 'Kuldak',
 'Fungal Beast',
 'Spore Carrier',
 'Maldar',
 'Monstrosity',
 'Thoon',
 'Blight Mage',
 'Blood Shaman',
 'Valeria the Just',
 'Arbiter of Peace',
 'Arbiter of Justice',
 'Albus the Wise',
 'Cassia the Gentle',
 'Titus the Devout',
 'Arbiter of War',
 'Attendant Priest',
 'Sunderved',
 'Wolfsbane Brute',
 'Shale Slinger',
 'Quen',
 'Torodin',
 'Varn',
 'Glory Caller',
 'Slagborn Fighter',
 'Balzar',
 'Ember Guard',
 'Blood Berserker',
 'Davura',
 'The Hell Forge',
 'Vulkør',
 'Ember Demon',
 'Oath Sworn',
 'Queen Maldaria',
 'Ember Archer',
 'Royal Guardian',
 'Fanessa',
 'Fire Drake',
 'Holleas',
 'Ember Beast',
 'Ember Mage',
 'Svara',
 'Frost Mage',
 'Ice Golem',
 'Jarmund',
 'Nadiana',
 'Ollag',
 'Bear Cavalry',
 'Ice Smith',
 'Parapet',
 'Krusk',
 'Wrench Rat',
 'Rust Rider',
 'Clink',
 'Silts',
 'Uncle Sparks',
 'Bug Biter',
 'Sand Scavenger',
 'Abua Shi',
 'Border Archer',
 'Lioness',
 'Makeinda Ru',
 'Miti Kyru',
 'Miti Mumway',
 'Rhinoceros',
 'Spirit Mage',
 'Selundar',
 'Shadow Mage',
 'Sage Ranger',
 'Lucian',
 'Samara',
 'Taliya',
 'Shadow Knight',
 'True Seeker',
 'Ava',
 'Aerie Sage',
 'Aerie Steward',
 'Gilford',
 'Rath',
 'Sairook',
 'Aerie Diver',
 'Aerie Sentinel',
 'Aerie Gate',
 'Mugglug',
 'Vinemancer',
 'Bog Shaman',
 'Blub',
 'Glurp',
 'Splack',
 'Bog Witch',
 'Mud Drudger',
 'Vine Gate',
 'Grognack',
 'Frost Shaman',
 'Tundra Fighter',
 'Ragnor',
 'Shonk',
 'Targan',
 'Smasher',
 'Tundra Charger',
 'Sera Eldwyn',
 'Temple Priest',
 'Citadel Knight',
 'Colleen Brighton',
 'Jacob Eldwyn',
 'Valna Stoutheart',
 'Citadel Archer',
 'Citadel Paladin',
 'Celeste',
 'Gem Dancer',
 'Spirit Voyager',
 'Boldo',
 'Gusto',
 'Tess',
 'Eager Adventurer',
 'Wayward Rogue',
 'Astral Waypoint']

['Tacullu',
 'Deceiver',
 'Mind Witch',
 'Gulldune',
 'Gwalark',
 'Kalal',
 'Wind Archer',
 'Wind Mage',
 'Sneeks',
 'Horde Slinger',
 'Beast Rider',
 'Blarf',
 'Smeg',
 'The Eater',
 'Clinger',
 'Horde Climber',
 'Vlox',
 'Guild Machinist',
 'Guild Sniper',
 'Admiral',
 'Dagger',
 'Sin-Sin',
 'Guild Gunner',
 'Rustside Bandit',
 'Baron Daemar',
 'Royal Warrior',
 'Devout Cultist',
 'Casimira',
 'Sekhmet',
 'Vesna Brana',
 'Doom Cultist',
 'Sacral Priest',
 'Praefectus Edia',
 'Citadel Advisor',
 'Mind Knight',
 'Dominus Katu',
 'Dominus Malunar',
 'Dominus Ovi',
 'Ancient Scholar',
 'Enigma Sage',
 'Ret-Talus',
 'Undead Carrier',
 'Undead Archer',
 'Dragos',
 'Elut-Bal',
 'Gul-Dass',
 'Hellfire Cultist',
 'Undead Warrior',
 'Kuldak',
 'Fungal Beast',
 'Spore Carrier',
 'Maldar',
 'Monstrosity',
 'Thoon',
 'Blight Mage',
 'Blood Shaman',
 'Valeria the Just',
 'Arbiter of Peace',
 'Arbiter of Justice',
 'Albus the Wise',
 'Cassia the Gentle',
 'Titus the Devout',
 'Arbiter of War',
 'At

In [100]:
card_names = ['Tacullu', 'Mind Control', 'Deceiver', 'Mind Witch', 'Gulldune', 'Gwalark', 'Kalal', 'Wind Archer',
 'Wind Mage', 'Blast', 'Hypnotic Call', 'Perplexing Tempest', 'Sneeks', 'Enrage the Horde', 'Horde Slinger', 'Beast Rider',
 'Blarf', 'Smeg', 'The Eater', 'Clinger', 'Horde Climber', 'Pile On', 'Sneak', 'Unrelenting', 'Vlox', 'Out of Shadows',
 'Guild Machinist', 'Guild Sniper', 'Admiral', 'Dagger', 'Sin-Sin', 'Guild Gunner', 'Rustside Bandit',
 'Hand Cannon', 'Moto Boots', 'Smoke Bomb', 'Baron Daemar', 'Convert', 'Royal Warrior', 'Devout Cultist',
 'Casimira', 'Sekhmet', 'Vesna Brana', 'Doom Cultist', 'Sacral Priest', 'Call Forth', 'Death Rite',
 'Transfuse', 'Praefectus Edia', 'Learn', 'Citadel Advisor', 'Mind Knight', 'Dominus Katu', 'Dominus Malunar',
 'Dominus Ovi', 'Ancient Scholar', 'Enigma Sage', 'Insight', 'Psychic Assault', 'Seek', 'Ret-Talus', 'Sacrificial Pyre',
 'Undead Carrier', 'Undead Archer', 'Dragos', 'Elut-Bal', 'Gul-Dass', 'Hellfire Cultist', 'Undead Warrior',
 'Blood Summon', 'Hellforged Weapon', 'Purge', 'Kuldak', 'Release Spores', 'Fungal Beast', 'Spore Carrier', 'Maldar',
 'Monstrosity', 'Thoon', 'Blight Mage', 'Blood Shaman', 'Demand', 'Stimshrooms', 'Symbiotic Healing', 'Valeria the Just',
 'Oblige', 'Arbiter of Peace', 'Arbiter of Justice', 'Albus the Wise', 'Cassia the Gentle', 'Titus the Devout',
 'Arbiter of War', 'Attendant Priest', 'Law of Liberty', 'Law of Loyalty', 'Law of Sanctity', 'Sunderved', 'Stronghold',
 'Wolfsbane Brute', 'Shale Slinger', 'Quen', 'Torodin', 'Varn', 'Glory Caller', 'Slagborn Fighter', 'Battle Leader',
 'Marshal Forces', 'Ramming Advance', 'Balzar', 'Volcanic Blast', 'Ember Guard', 'Blood Berserker', 'Davura',
 'The Hell Forge', 'Vulkør', 'Ember Demon', 'Oath Sworn', 'Hellforged Axe', 'Hellforged Chains', 'Willpower',
 'Queen Maldaria', 'Spirit of the Phoenix', 'Ember Archer', 'Royal Guardian', 'Fanessa', 'Fire Drake', 'Holleas',
 'Ember Beast', 'Ember Mage', 'Blinding Flare', 'Burn', 'Divine Retribution', 'Svara', 'Ice Ram', 'Frost Mage',
 'Ice Golem','Jarmund','Nadiana','Ollag', 'Bear Cavalry', 'Ice Smith', 'Glacial Shift', 'Ice Repair', 'Parapet',
 'Krusk', 'Detonate', 'Wrench Rat', 'Rust Rider', 'Clink', 'Silts', 'Uncle Sparks', 'Bug Biter', 'Sand Scavenger',
 'Mortar Upgrade', 'Thruster Upgrade', 'Turret Upgrade', 'Abua Shi', 'Chant of Power', 'Border Archer', 'Lioness',
 'Makeinda Ru', 'Miti Kyru', 'Miti Mumway', 'Rhinoceros', 'Spirit Mage', 'Chant of Entangling', 'Chant of Growth',
 'Chant of Weaving','Selundar','Into Darkness','Shadow Mage','Sage Ranger', 'Lucian', 'Samara', 'Taliya', 'Shadow Knight',
 'True Seeker', 'Glinting Speed', 'Onyx Tome', 'Shadow Pulse', 'Ava', 'Gathering Song', 'Aerie Sage', 'Aerie Steward',
 'Gilford', 'Rath', 'Sairook', 'Aerie Diver', 'Aerie Sentinel', 'Aerie Gate', 'Battle Song', 'Sky Assault', 'Mugglug',
 'Emerge', 'Vinemancer', 'Bog Shaman', 'Blub', 'Glurp', 'Splack', 'Bog Witch', 'Mud Drudger', 'Regenerate', 'Sow',
 'Vine Gate', 'Grognack', 'For Glory', 'Frost Shaman', 'Tundra Fighter', 'Ragnor', 'Shonk', 'Targan', 'Smasher',
 'Tundra Charger', 'Brutal Force', 'Freeze', 'Primal Fury', 'Sera Eldwyn', 'Holy Judgment', 'Temple Priest', 'Citadel Knight',
 'Colleen Brighton', 'Jacob Eldwyn', 'Valna Stoutheart', 'Citadel Archer', 'Citadel Paladin', 'Guarding Spirits',
 'Mass Heal', 'Renewed Hope', 'Celeste', 'Coordinate', 'Gem Dancer', 'Spirit Voyager', 'Boldo', 'Gusto', 'Tess',
 'Eager Adventurer', 'Wayward Rogue', 'Astral Waypoint', 'Seek the Horizon', 'Slip By']

In [ ]:
cards_that_go_under = ['Hand Cannon', 'Moto Boots', 'Hellforged Weapon', 'Hellforged Axe', 'Hellforged Chains',
 'Ice Smith', 'Wrench Rat', 'Mortar Upgrade', 'Thruster Upgrade', 'Turret Upgrade']

In [ ]:
card_abilities = ['Mind Capture', 'Stupify', 'Engage', 'Mimic', 'Telepathic Command', 'Flight', 'Levitate', 'Greater Push',
 'Steadfast', 'Far Shot', 'Swift', 'Push', 'Sly', 'Relentless', 'Charge', 'Blood Runes', 'Imbued Strength',
 'Magic Junkie', 'Feed the Eater', 'Immobile', 'Cling', 'Climb', 'Gang Up', 'Master of the Art', 'Blindside',
 'Versatile', 'Steady Shot', 'Backstab', 'Bounty', 'Infiltrate', 'Cover Fire', 'Heist', 'Power Shot',
 'Burst of Speed', 'Blood Drain', 'Faithful Servants', 'Blood Begotten', 'Undying Devotion', 'Blood Thrall',
 'Blood Hunger', 'Blood Ward', 'Curse', 'Protector', 'Blood Fervor', 'Kinetic Siphon', 'Persist',
 'Intel', 'Warn', 'Invoke Fear', 'Shove', 'Resolute', 'Punish', 'Mastermind', 'Wise', 'Analyze', 'Exploit',
 'Raise the Dead', 'Soulless', 'Infect', 'Soul Shift', 'Life Drain', 'Summoned By Fire', 'Wrath',
 'Immolate', 'Blood Fury', 'Cursed', 'Blood Bloom', 'Parasitic', 'Rupture', 'Fungal Transformation',
 'Corrupt', 'Final Form', 'Blood Imbue', 'Transfer', 'Dictate', 'Greater Engage', 'Tough', 'Law of Fortitude',
 'Atone', 'Law of Atonement', 'Annul', 'Healing Aura', 'Protect', 'Strong', 'Law of Strength', 'Attend',
 'Invigorate', 'Commander', 'Slow', 'Ram', 'Coordinated Shot', 'Lightning Strike', 'Trample',
 'Form Up', 'Call to Arms', 'Forceful Attack', 'Burning Rage', 'Blaze Step', 'Wild Swing', 'Disciple',
 'Coordinated Strike', 'Inanimate', 'Work the Forge', 'Imbued Spirit', 'Born of Fire', 'Dark Pact', 'Blood Bond',
 'Too Strong', 'Smolder', 'Reach', 'Iron Will', 'Call Protection', 'Save the Queen', 'Quick Shot', 'Riposte',
 'Fire Breath', 'Fiery Summon', 'Ignite', 'Wildfire', 'Guarded', 'Aura of the Phoenix', 'Structural Shift',
 'Frost Bolt', 'Living Gate', 'Mobile Structure', 'Momentum', 'Ice Shards', 'Greater Frost Bolt', 'Cold Snap',
 'Frost Axe', 'Carry', 'Rig Explosives', 'Burn Hot', 'Repair', 'Cunning', 'Top Wrench', 'Crash', 'Scrap',
 'Explosive Shell', 'Thrust', 'Gunner', 'Spirit Bond', 'Prepare', 'Swift Shot', 'Imbued Life', 'Inspire',
 'Withdraw', 'Imbued Speed', 'Charged', 'Commune With Spirits', 'Blood Magic', 'Return to Shadow', 'Forbidden Knowledge',
 'Piercing Light', 'Rush', 'Dark Prophecy', 'Judgment', 'Inevitable Doom', 'Rend the Veil', 'Shadow Summon',
 'Death Pact', 'Aggressive Attack', 'Feint', 'Flash Step', 'Take to the Skies', 'Gather the Flock',
 'Aerie Support', 'Worthy', 'Bully', 'Prey', 'Aerie Captain', 'Dive In', 'Sudden Strike', 'Stalwart', 'Blitz',
 'Aerial Shot', 'Reap', 'Far Strike', 'Grow', 'Vine Pull', 'Entwine', 'Germinate', 'Tireless', 'Augment', 'Prune',
 'Restore', 'Influence', 'Reckless Strike', 'Unwieldy Magic', 'Frenzy', 'Wild Strike', 'Piercing Frost',
 'Sluggish', 'Blood Rush', "The Citadel's Might", 'Heal', 'Divine Shield', 'Radiant Shot', 'Guidance',
 'Citadel Champion', 'Arrow of Light', 'Intercession', 'Well Traveled', 'Footwork', 'Spirit Leap',
 'Agile', 'Favorable Winds', 'Illusions', 'Well-Prepared', 'Embark', 'Sneak Attack']

In [ ]:
card_abilities = 

In [101]:
len(card_names)

240

In [105]:
len(card_ability_list)

262

In [108]:
unique_card_ability_list = []
for i in card_ability_list:
    if i not in unique_card_ability_list:
        unique_card_ability_list.append(i)
len(unique_card_ability_list)

211

In [109]:
unique_card_ability_list

['Mind Capture',
 'Stupify',
 'Engage',
 'Mimic',
 'Telepathic Command',
 'Flight',
 'Levitate',
 'Greater Push',
 'Steadfast',
 'Far Shot',
 'Swift',
 'Push',
 'Sly',
 'Relentless',
 'Charge',
 'Blood Runes',
 'Imbued Strength',
 'Magic Junkie',
 'Feed the Eater',
 'Immobile',
 'Cling',
 'Climb',
 'Gang Up',
 'Master of the Art',
 'Blindside',
 'Versatile',
 'Steady Shot',
 'Backstab',
 'Bounty',
 'Infiltrate',
 'Cover Fire',
 'Heist',
 'Power Shot',
 'Burst of Speed',
 'Blood Drain',
 'Faithful Servants',
 'Blood Begotten',
 'Undying Devotion',
 'Blood Thrall',
 'Blood Hunger',
 'Blood Ward',
 'Curse',
 'Protector',
 'Blood Fervor',
 'Kinetic Siphon',
 'Persist',
 'Intel',
 'Warn',
 'Invoke Fear',
 'Shove',
 'Resolute',
 'Punish',
 'Mastermind',
 'Wise',
 'Analyze',
 'Exploit',
 'Raise the Dead',
 'Soulless',
 'Infect',
 'Soul Shift',
 'Life Drain',
 'Summoned By Fire',
 'Wrath',
 'Immolate',
 'Blood Fury',
 'Cursed',
 'Blood Bloom',
 'Parasitic',
 'Rupture',
 'Fungal Transformation'

In [98]:
under_card_list

['Hand Cannon',
 'Moto Boots',
 'Hellforged Weapon',
 'Hellforged Axe',
 'Hellforged Chains',
 'Ice Smith',
 'Wrench Rat',
 'Mortar Upgrade',
 'Thruster Upgrade',
 'Turret Upgrade']

In [ ]:
# for multiplayer where you don't get to choose opponent
def get_enemy_summoner(driver):

In [ ]:
unit_classes = ['Summoner', 'Structure', 'Common', 'Champion']

In [ ]:
# zoom-card-extras-event-phase
# added draw phase just to be safe and future proof in case they come out with a draw phase event 15511
event_phases = ['Summon Phase', 'Move Phase', 'Build Phase', 'Attack Phase', 'Magic Phase', 'Draw Phase']

In [ ]:
# observation, reward, done, and info?

In [ ]:
# gets the current state of the board, cards in hand, discarded cards, active events, cards in deck, etc.
def get_board_state()

In [ ]:
# possible actions that can be taken at any given point
def get_available_actions()

In [ ]:
# choose an action, evaluating them?? (need to refresh on reinforcement learning here)
def evaluate_actions()